In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/news/

Mounted at /content/drive


In [7]:
import pandas as pd
import numpy as np
from numpy import array
import nltk
from nltk.corpus import stopwords
import re
from sklearn.model_selection import train_test_split

nltk.download('stopwords')

/content/drive/MyDrive/news


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:
imdb_reviews = pd.read_csv("imdb_reviews.csv")
# safety check
imdb_reviews.shape

(50000, 2)

preprocessing

In [16]:
def preprocess(s):
    s = s.lower()

    s = re.compile(r'<[^>]+>').sub('', s)

    # remove punctuations/numbers
    s = re.sub('[^a-zA-Z]', ' ', s)

    # remove single letters
    s = re.sub(r"\s+[a-zA-Z]\s+", ' ', s)

    # remove places with more than one space
    s = re.sub(r'\s+', ' ', s)

    # remove nltk stopwords (regex makes it go wayyy faster than the line below)
    # s = ' '.join([word for word in s.split() if word not in (stopwords.words('english'))])
    s = re.compile(r'\b(' + r'|'.join(stopwords.words('english')) + r')\b\s*').sub('', s)

    return s

In [14]:
print(preprocess("i use ` rust for BLLLLLAZINGLY fast code    ... its amazing"))

use rust blllllazingly fast code amazing


In [17]:
# preprocessing all reviews
preprocessed_text = []

for review in list(imdb_reviews['review']):
    preprocessed_text.append(preprocess(review))

print(preprocessed_text[1])

wonderful little production filming technique unassuming old time bbc fashion gives comforting sometimes discomforting sense realism entire piece actors extremely well chosen michael sheen got polari voices pat truly see seamless editing guided references williams diary entries well worth watching terrificly written performed piece masterful production one great master comedy life realism really comes home little things fantasy guard rather use traditional dream techniques remains solid disappears plays knowledge senses particularly scenes concerning orton halliwell sets particularly flat halliwell murals decorating every surface terribly well done 


In [19]:
# converting each review's "sentiment" column value to 1 and 0 instead of "positive" and "negative"
imdb_sentiments = imdb_reviews['sentiment']
sentiments = []

for sentiment in imdb_sentiments:
    if sentiment == "positive":
        sentiments.append(1)
    else:
        sentiments.append(0)

sentiments = np.array(sentiments)

print(sentiments[1])

1


In [20]:
# spliting data into training and test set data for the model
X_train, X_test, y_train, y_test = train_test_split(preprocessed_text, sentiments, test_size=0.25, random_state=42)